In [ ]:
# Attendence System - I
# Encode and Save Known Faces

import cv2
import numpy as np
import mediapipe as mp
import os

cap = cv2.VideoCapture(0)
mp_face = mp.solutions.face_mesh
face_mesh = mp_face.FaceMesh(refine_landmarks=True,static_image_mode=True)

def extract_face_incoding(img):
    h,w,_ = img.shape
    rgb = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    res = face_mesh.process(rgb)

    if res.multi_face_landmarks:
        for landmarks in res.multi_face_landmarks:
            face_encoding = []
            for lm in landmarks.landmark:
                x,y = int(lm.x*w),int(lm.y*h)
                face_encoding.append([x,y])
            return np.array(face_encoding)
    return None


while cap.isOpened():
    r,f = cap.read()
    if r==False:
        break
    f = cv2.flip(f,1)
    cv2.imshow("Register Face",f)
    
    key = cv2.waitKey(1) & 0xff 
    if key == ord('s'):
        name = input("Enter Name")
        encoding = extract_face_incoding(f)
        if encoding is not None:
            np.save(f'encoding/{name}.npy',encoding)
            print(f'Saved encoding for {name}')
        else:
            print("Face not detected. Try Again")

    elif key == ord('d'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
# Attendence System - II
# Detecting Face For attendence

import cv2
import os
import numpy as np
import mediapipe as mp
from scipy.spatial import distance as dist
from datetime import datetime


known_face = {}
for file in os.listdir("encoding"):
    if file.endswith(".npy"):
        name = file.split(".")[0]
        encoding = np.load(f'encoding/{file}')
        known_face[name]=encoding

mp_face = mp.solutions.face_mesh
face_mesh = mp_face.FaceMesh(static_image_mode=False,refine_landmarks=True)
mp_draw = mp.solutions.drawing_utils

marked_present = set()
attendence = {}

def mark_attendence(name):
    now = datetime.now().strftime("%H:%M:%S")
    attendence[name] = True

def calculateSimilarity(face1,face2):
    if len(face1)!=len(face2):
        return float('inf')
    return np.mean([dist.euclidean(p1,p2) for p1,p2 in zip(face1,face2)])

threshold = 20


cap = cv2.VideoCapture(0)
print("Info Running Attendence System.....")

while cap.isOpened():
    r,f = cap.read()
    if r==False:
        break
    f = cv2.flip(f,1)
    h,w,_ = f.shape
    rgb = cv2.cvtColor(f,cv2.COLOR_BGR2RGB)
    res = face_mesh.process(rgb)


    status = "Face not Detected"
    color = (0,0,255)

    if res.multi_face_landmarks:
        for landmarks in res.multi_face_landmarks:
            live_encoding = []
            for lm in landmarks.landmark:
                x,y = int(lm.x*w),int(lm.y*h)
                live_encoding.append([x,y])
                cv2.circle(f,(x,y),1,(0,255,0),-1)
            matched_name = None
            min_dist = float('inf')

            for name,save_encoding in known_face.items():
                dist_val = calculateSimilarity(save_encoding,live_encoding)
                if dist_val<threshold and dist_val<min_dist:
                    matched_name=name
                    min_dist=dist_val
            if matched_name:
                status = f'{matched_name} - Present'
                color = (0,255,0)
                if matched_name not in marked_present:
                    mark_attendence(matched_name)
                    marked_present.add(matched_name)
            mp_draw.draw_landmarks(f,landmarks,mp_face.FACEMESH_CONTOURS,mp_draw.DrawingSpec(color=(0,255,0),thickness=1,circle_radius=1),mp_draw.DrawingSpec(color=(0,0,255),thickness=1))
    cv2.putText(f,f"{status} : {dist_val}",(10,30),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,color,2)

    cv2.imshow("Attendence Page",f)
    if cv2.waitKey(1) & 0xff == ord('d'):
        break

cap.release()
cv2.destroyAllWindows()